In [1]:
import os

In [2]:
%pwd

'e:\\ML_Projects\\Kidney-Disease-Classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'e:\\ML_Projects\\Kidney-Disease-Classification'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from TumorClassifier.constants import *
from TumorClassifier.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
        
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )
        
        return data_ingestion_config

In [8]:
import os
import zipfile
import gdown
from TumorClassifier import logger

In [9]:
class DataIngestion:
    def __init__ (self, config: DataIngestionConfig):
        self.config = config
        
    def download_file(self)->str:
        
        """
        To fetch data from URL
        """
        
        try:
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok= True)
            logger.info(f"Downloading data from {dataset_url} into file")
            
            file_id = dataset_url.split("/")[-2]
            prefix = "https://drive.google.com/uc?/export=download&id="
            gdown.download(prefix + file_id, zip_download_dir)
            
            logger.info(f"Downloaed data from {dataset_url}")
            
        except Exception as e:
            raise e
        
        
    def extract_zip_file(self):
        """
        Extracts the zip file into the data directory
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-11-26 00:10:15,769: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-11-26 00:10:15,777: INFO: common: yaml file: params.yaml loaded successfully]
[2023-11-26 00:10:15,779: INFO: common: created directory at: artifacts]
[2023-11-26 00:10:15,782: INFO: 396214572: Downloading data from https://drive.google.com/file/d/18TT7KtXfeGSmdF2dgE_ygewCTWqmki-R/view?usp=sharing into file]


Downloading...
From: https://drive.google.com/uc?/export=download&id=18TT7KtXfeGSmdF2dgE_ygewCTWqmki-R
To: e:\ML_Projects\Kidney-Disease-Classification\artifacts\data_ingestion\data.zip
100%|██████████| 25.2M/25.2M [00:13<00:00, 1.81MB/s]

[2023-11-26 00:10:32,705: INFO: 396214572: Downloaed data from https://drive.google.com/file/d/18TT7KtXfeGSmdF2dgE_ygewCTWqmki-R/view?usp=sharing]
